# 1. Inspect the data from the rommate

## Load roommate data

In [1]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/SRR1705851.fastq.gz

--2019-11-25 23:01:06--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/001/SRR1705851/SRR1705851.fastq.gz
           => «SRR1705851.fastq.gz»
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.192.7
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.7|:21... соединение установлено.
Выполняется вход под именем anonymous … Выполнен вход в систему!
==> SYST ... готово.  ==> PWD ... готово.
==> TYPE I ... готово.   ==> CWD (1) /vol1/fastq/SRR170/001/SRR1705851 ... готово.
==> SIZE SRR1705851.fastq.gz ... 19217176
==> PASV ... готово.  ==> RETR SRR1705851.fastq.gz ... готово.
Размер (байт): 19217176 (18M) (не достоверно)

SRR1705851.fastq.gz 100%[===================>]  18,33M  3,75MB/s    за 7,0s    

2019-11-25 23:01:15 (2,62 MB/s) - «SRR1705851.fastq.gz» сохранён [19217176]



In [3]:
!gunzip SRR1705851.fastq.gz

In [65]:
!head -10 SRR1705851.fastq

@SRR1705851.1 1/1
TTCGTGATTGTTTTCACTATCGTTCCGTTTGGCACTGCATGGTGCCCAAGGCACAGCGTTGCCGTGCTGTTGTCATTTCCAGGAAGTTTTTGAGCGAAAACCAGACATAGAATGTAGCTCAAAGCAATGATAGTCTTCATGGTTAATAG
+
,<==<<<<A@@@@@@@EEE;CEE+AC>EC;>EFFDC@=A@AE999DDD>>@E777EE75C>EF>EDEEFFFF--AE>EDEEEED=C-58AE=<D=<<DD=D9CDD@EEDED@DEDDE*9;@DDED@@@7@E*;*888@*8;@8@;;@@E
@SRR1705851.2 2/1
NATTAACCATGAAGACTATCATTGCTTTGAGCTACATTCTATGTCTGGTTTTCGCTCAAAAACTTCCTGGAAATGACAACAGCACGGCAACGCTGTGCCTTGGGCACCATGCAGTGCCAAACGGAACGATAGTGAAAACAATCACGAATGA
+
#5<???BBEEEDEDDDGGGGGGIIIIIIIIIIIIIIIIIIIIIHIIIIFHHIIHHHHHIIIIHIIIIIIIHIIIIIIIIIIIIIIHHHHHHHHHHEHHHHHFFHHHHHHFFHHGFGGGGGGGGGGGGGEEEGCEEGGGGGEEGGGGCGEGG
@SRR1705851.3 3/1
GTTTGGCACTGCATGGTGCCCAAGGCACAGCGTTGCCGTGCTGTTGTCATTTCCAGGAAGTTTTTGAGCGAAAACCAGACATAGAATGTAGCTCAAAGCAATGATAGTCTTCATGGTTAATAG


#### Average read lenght

In [72]:
!awk '{if(NR%4==2) {count++; bases += length} } END{print bases/count}' SRR1705851.fastq

147,148


#### Number of reads (has to be divided by 4)

In [73]:
!wc -l SRR1705851.fastq

 1433060 SRR1705851.fastq


## Loading reference data

In [32]:
!efetch -db nuccore -id KF848938.1 -format fasta> influenza_h_ref.fasta

In [46]:
!bwa index influenza_h_ref.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index influenza_h_ref.fasta
[main] Real time: 0.008 sec; CPU: 0.021 sec


#### Calculating number of bases

In [78]:
!awk '{if(NR>1) {bases += length} } END{print bases}' influenza_h_ref.fasta

1665


### Coverage = (#reads * mean read length) / #bases in reference

In [80]:
round(1433060 * 150 / 4 / 1665, 0)

32276.0

# 2. Align your roommate’s data to the reference sequence

In [47]:
!bwa mem influenza_h_ref.fasta SRR1705851.fastq | samtools view -S -b - | samtools sort - -o roommate_sorted.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 68388 sequences (10000129 bp)...
[M::process] read 67628 sequences (10000233 bp)...
[M::mem_process_seqs] Processed 68388 reads in 1.849 CPU sec, 1.788 real sec
[M::process] read 67698 sequences (10000046 bp)...
[M::mem_process_seqs] Processed 67628 reads in 1.501 CPU sec, 1.362 real sec
[M::process] read 67652 sequences (10000169 bp)...
[M::mem_process_seqs] Processed 67698 reads in 1.952 CPU sec, 1.819 real sec
[M::process] read 68072 sequences (10000295 bp)...
[M::mem_process_seqs] Processed 67652 reads in 1.904 CPU sec, 1.769 real sec
[M::process] read 18827 sequences (2716992 bp)...
[M::mem_process_seqs] Processed 68072 reads in 1.629 CPU sec, 1.538 real sec
[M::mem_process_seqs] Processed 18827 reads in 0.812 CPU sec, 0.753 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem influenza_h_ref.fasta SRR1705851.fastq
[main] Real time: 9.649 sec; CPU: 9.782 sec


In [48]:
!samtools index roommate_sorted.bam

In [52]:
!samtools flagstat roommate_sorted.bam

361349 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
3084 + 0 supplementary
0 + 0 duplicates
361116 + 0 mapped (99.94% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


# 3. Look for common variants with VarScan

## Prepare variants, -d 35000, since coverage is ~ 32000

In [81]:
!samtools mpileup -d 35000 -f influenza_h_ref.fasta roommate_sorted.bam >  my.mpileup

[mpileup] 1 samples in 1 input files


## VarScan results with min frequency 0.95

In [67]:
!java -jar VarScan.v2.4.4.jar  mpileup2snp my.mpileup --min-var-freq 0.95 --variants --output-vcf 1 > roommate_varscan_95_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.95
Min avg qual:	15
P-value thresh:	0.01
Reading input from my.mpileup
1665 bases in pileup file
5 variant positions (5 SNP, 0 indel)
0 were failed by the strand-filter
5 variant positions reported (5 SNP, 0 indel)


In [68]:
!cat roommate_varscan_95_results.vcf | awk 'NR>24 {print $1, $2, $4, $5}'

KF848938.1 72 A G
KF848938.1 117 C T
KF848938.1 774 T C
KF848938.1 999 C T
KF848938.1 1260 A C


### Analysing varscan results in igv

There 5 snps:
1) ACA -> ACG, no amino acid change
2) GCC -> GCU, no amino acid change
3) UUU -> UUC, no amino acid change
4) GGC -> GGU, no amino acid change
5) CUA -> CUC, no amino acid change

# 4. Look for rare variants with VarScan

## VarScan results with min frequency 0.001

In [1]:
!java -jar VarScan.v2.4.4.jar  mpileup2snp my.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > roommate_varscan_001_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from my.mpileup
1665 bases in pileup file
20 variant positions (18 SNP, 2 indel)
0 were failed by the strand-filter
18 variant positions reported (18 SNP, 0 indel)


In [8]:
!cat roommate_varscan_001_results.vcf | awk 'NR>24 { split($10,x,":"); print $1, $2, $4, $5, x[7]}'

KF848938.1 72 A G 99,96%
KF848938.1 117 C T 99,82%
KF848938.1 254 A G 0,17%
KF848938.1 307 C T 0,95%
KF848938.1 340 T C 0,17%
KF848938.1 389 T C 0,22%
KF848938.1 722 A G 0,22%
KF848938.1 744 A G 0,18%
KF848938.1 774 T C 99,96%
KF848938.1 802 A G 0,24%
KF848938.1 915 T C 0,2%
KF848938.1 999 C T 99,86%
KF848938.1 1043 A G 0,19%
KF848938.1 1086 A G 0,21%
KF848938.1 1213 A G 0,22%
KF848938.1 1260 A C 99,94%
KF848938.1 1280 T C 0,18%
KF848938.1 1458 T C 0,84%


# 5. Inspect and align the control sample sequencing data

SRR1705858, SRR1705859, SRR1705860

In [12]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/008/SRR1705858/SRR1705858.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/009/SRR1705859/SRR1705859.fastq.gz  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/000/SRR1705860/SRR1705860.fastq.gz 

--2019-11-26 22:53:20--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/008/SRR1705858/SRR1705858.fastq.gz
           => «SRR1705858.fastq.gz»
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.192.7
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.7|:21... соединение установлено.
Выполняется вход под именем anonymous … Выполнен вход в систему!
==> SYST ... готово.  ==> PWD ... готово.
==> TYPE I ... готово.   ==> CWD (1) /vol1/fastq/SRR170/008/SRR1705858 ... готово.
==> SIZE SRR1705858.fastq.gz ... 14782243
==> PASV ... готово.  ==> RETR SRR1705858.fastq.gz ... готово.
Размер (байт): 14782243 (14M) (не достоверно)

SRR1705858.fastq.gz 100%[===================>]  14,10M  2,37MB/s    за 8,2s    

2019-11-26 22:53:30 (1,71 MB/s) - «SRR1705858.fastq.gz» сохранён [14782243]

--2019-11-26 22:53:30--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR170/009/SRR1705859/SRR1705859.fastq.gz
           => «SRR1705859.fastq.gz»
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.

In [13]:
!gunzip SRR1705858.fastq.gz SRR1705859.fastq.gz SRR1705860.fastq.gz

In [15]:
!wc -l SRR1705858.fastq SRR1705859.fastq SRR1705860.fastq

 1026344 SRR1705858.fastq
  933308 SRR1705859.fastq
  999856 SRR1705860.fastq
 2959508 total


### New files coverage

In [20]:
round(10 ** 6 / 4 * 150 / 1665, 0)

22523.0

## Align control files

#### First

In [21]:
!bwa mem influenza_h_ref.fasta SRR1705858.fastq | samtools view -S -b - | samtools sort - -o control_1_sorted.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 67448 sequences (10000210 bp)...
[M::process] read 67230 sequences (10000094 bp)...
[M::mem_process_seqs] Processed 67448 reads in 1.360 CPU sec, 1.296 real sec
[M::process] read 67100 sequences (10000113 bp)...
[M::mem_process_seqs] Processed 67230 reads in 1.606 CPU sec, 1.497 real sec
[M::process] read 54808 sequences (8118313 bp)...
[M::mem_process_seqs] Processed 67100 reads in 1.601 CPU sec, 1.523 real sec
[M::mem_process_seqs] Processed 54808 reads in 1.324 CPU sec, 1.257 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem influenza_h_ref.fasta SRR1705858.fastq
[main] Real time: 6.484 sec; CPU: 6.048 sec


In [44]:
!bwa mem influenza_h_ref.fasta SRR1705859.fastq | samtools view -S -b - | samtools sort - -o control_2_sorted.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 67476 sequences (10000274 bp)...
[M::process] read 67236 sequences (10000039 bp)...
[M::mem_process_seqs] Processed 67476 reads in 1.339 CPU sec, 1.284 real sec
[M::process] read 67216 sequences (10000283 bp)...
[M::mem_process_seqs] Processed 67236 reads in 1.552 CPU sec, 1.435 real sec
[M::process] read 31399 sequences (4635971 bp)...
[M::mem_process_seqs] Processed 67216 reads in 1.492 CPU sec, 1.415 real sec
[M::mem_process_seqs] Processed 31399 reads in 0.876 CPU sec, 0.831 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem influenza_h_ref.fasta SRR1705859.fastq
[main] Real time: 5.575 sec; CPU: 5.379 sec


In [29]:
!bwa mem influenza_h_ref.fasta SRR1705860.fastq | samtools view -S -b - | samtools sort - -o control_3_sorted.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 67356 sequences (10000126 bp)...
[M::process] read 67208 sequences (10000122 bp)...
[M::mem_process_seqs] Processed 67356 reads in 1.352 CPU sec, 1.308 real sec
[M::process] read 67010 sequences (10000072 bp)...
[M::mem_process_seqs] Processed 67208 reads in 1.635 CPU sec, 1.576 real sec
[M::process] read 48390 sequences (7170166 bp)...
[M::mem_process_seqs] Processed 67010 reads in 1.632 CPU sec, 1.633 real sec
[M::mem_process_seqs] Processed 48390 reads in 1.254 CPU sec, 1.219 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem influenza_h_ref.fasta SRR1705860.fastq
[main] Real time: 6.505 sec; CPU: 6.017 sec


In [41]:
!samtools index control_1_sorted.bam control_2_sorted.bam control_3_sorted.bam

In [46]:
!samtools flagstat control_2_sorted.bam

233451 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
124 + 0 supplementary
0 + 0 duplicates
233375 + 0 mapped (99.97% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


### Preparing mplilup files

In [37]:
!samtools mpileup -d 25000 -f influenza_h_ref.fasta control_1_sorted.bam > control_1.mpileup

[mpileup] 1 samples in 1 input files


In [47]:
!samtools mpileup -d 25000 -f influenza_h_ref.fasta control_2_sorted.bam > control_2.mpileup

[mpileup] 1 samples in 1 input files


In [39]:
!samtools mpileup -d 25000 -f influenza_h_ref.fasta control_3_sorted.bam > control_3.mpileup

[mpileup] 1 samples in 1 input files


## Preparing vcf files

In [48]:
!java -jar VarScan.v2.4.4.jar  mpileup2snp control_1.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > control_1_varscan.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from control_1.mpileup
1665 bases in pileup file
56 variant positions (56 SNP, 0 indel)
1 were failed by the strand-filter
55 variant positions reported (55 SNP, 0 indel)


In [49]:
!java -jar VarScan.v2.4.4.jar  mpileup2snp control_2.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > control_2_varscan.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from control_2.mpileup
1665 bases in pileup file
54 variant positions (54 SNP, 0 indel)
2 were failed by the strand-filter
52 variant positions reported (52 SNP, 0 indel)


In [50]:
!java -jar VarScan.v2.4.4.jar  mpileup2snp control_3.mpileup --min-var-freq 0.001 --variants --output-vcf 1 > control_3_varscan.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.001
Min avg qual:	15
P-value thresh:	0.01
Reading input from control_3.mpileup
1665 bases in pileup file
61 variant positions (61 SNP, 0 indel)
0 were failed by the strand-filter
61 variant positions reported (61 SNP, 0 indel)


## Preparing tables from vcf

In [60]:
!cat control_1_varscan.vcf | awk 'NR>24 { split($10,x,":"); print $1, $2, $4, $5, x[7]}' > control_1_results.csv

In [61]:
!cat control_2_varscan.vcf | awk 'NR>24 { split($10,x,":"); print $1, $2, $4, $5, x[7]}' > control_2_results.csv

In [62]:
!cat control_3_varscan.vcf | awk 'NR>24 { split($10,x,":"); print $1, $2, $4, $5, x[7]}' > control_3_results.csv

In [63]:
!cat roommate_varscan_001_results.vcf | awk 'NR>24 { split($10,x,":"); print $1, $2, $4, $5, x[7]}' > roomate_results.csv

## Analyse vcf results

availible at https://docs.google.com/spreadsheets/d/1hUd-mcM3aaoY02K1wQKTicjBTVq-7eQ7DHTz4SnSguc/edit?usp=sharing

### Statistical analysis has shown that mutations at positions 307 and 1458 are significant

307: CCG -> UCG (P)

1458: UAU -> UAC (Y) no amino acid change

According to epitope locations listed in Munoz et al, residue 307 is part of epitope C.
It is one of three regions (A-C) that are important for recognition by antibodies.